In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import requests
from urllib.parse import urlparse
from tqdm import tqdm
import json
import urllib.request
from pandas.io.json import json_normalize

In [2]:
def add_xy_points(data, x='x', y='y', buffer=500, envelope=True):
    geometry = gpd.points_from_xy(x=data[x], y=data[y])
    data = gpd.GeoDataFrame(data, geometry=geometry)  # DataFrame에 Point 추가
    data.crs = {'init':'epsg:4326'}
    data = data.to_crs({'init':'epsg:5179'})
    
    if buffer:
        if envelope==True:
            buffers = gpd.GeoSeries(data.geometry.buffer(buffer).envelope)
        else:
            buffers = gpd.GeoSeries(data.geometry.buffer(buffer))
    
        data['centroid_geometry'] = data['geometry']
        data['geometry'] = buffers
        
    return data

In [39]:
points = pd.read_csv('gwang.csv')

In [48]:
pro_points = add_xy_points(points, x='경도', y='위도', buffer=10, envelope=False)
pro_points = pro_points[pro_points['가맹점명']!='GS25 광안해변점']
pro_points = pro_points[pro_points['가맹점명']!='GS25 광안리중앙점']
pro_points = pro_points[pro_points['가맹점명']!='황금어장']
pro_points = pro_points[pro_points['가맹점명']!='정원해물탕']
pro_points

C:\Users\lynn1\anaconda3\envs\data-analysis\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\lynn1\anaconda3\envs\data-analysis\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,Unnamed: 0,순번,가맹점명,도로명,지번,위도,경도,데이터기준일자,geometry,centroid_geometry
0,223,224,명성초장집,부산 수영구 광안해변로15번길 77 (남천동),NaN,35.139159,129.109044,2022-12-22,"POLYGON ((1146595.427 1683845.297, 1146595.379...",POINT (1146585.427 1683845.297)
1,266,267,써브웨이광안비치점,"부산 수영구 광안해변로 239 1층 (민락동,광안샤인빌딩)",NaN,35.154777,129.120110,2022-12-22,"POLYGON ((1147575.509 1685593.942, 1147575.460...",POINT (1147565.509 1685593.942)
2,369,370,와바모텔,부산 수영구 광안해변로279번길 30 (민락동),NaN,35.156747,129.125494,2022-12-22,"POLYGON ((1148062.306 1685820.348, 1148062.258...",POINT (1148052.306 1685820.348)
3,374,375,쉼표,"부산 수영구 광안해변로 314 B동 101호 상가 (민락동, 진로비치아파트)",NaN,35.156907,129.127592,2022-12-22,"POLYGON ((1148253.166 1685841.302, 1148253.118...",POINT (1148243.166 1685841.302)
4,440,441,컴포즈커피 민락점,"부산 수영구 광안해변로 311 ,1층 117호 (민락동,서희스타힐스센텀프리모)",NaN,35.157332,129.127433,2022-12-22,"POLYGON ((1148237.939 1685888.218, 1148237.890...",POINT (1148227.939 1685888.218)
...,...,...,...,...,...,...,...,...,...,...
867,126006,126007,(주)2022부산기념품10선협의회 여가 거가 광,부산 수영구 광안해변로 219 1층 (광안동),NaN,35.153774,129.118516,2022-12-22,"POLYGON ((1147432.101 1685480.240, 1147432.053...",POINT (1147422.101 1685480.240)
868,126123,126124,지에스(GS)25 광안리중앙점,부산 수영구 광안해변로 195 1층 (광안동),NaN,35.151743,129.116809,2022-12-22,"POLYGON ((1147280.265 1685252.514, 1147280.217...",POINT (1147270.265 1685252.514)
869,126153,126154,삼천포,"부산 수영구 광안해변로344번길 17-9 101호 (민락동,마린리치2 )",NaN,35.155215,129.131202,2022-12-22,"POLYGON ((1148585.032 1685658.993, 1148584.984...",POINT (1148575.032 1685658.993)
870,126654,126655,청사옥,"부산 수영구 광안해변로344번길 17-5 4층 401호 (민락동,마린리치)",NaN,35.155181,129.131025,2022-12-22,"POLYGON ((1148569.053 1685654.914, 1148569.005...",POINT (1148559.053 1685654.914)


In [49]:
pro_points.explore(color='red')